In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.datasets import cifar10

In [2]:
(xTrain, yTrain), (xTest, yTest) = cifar10.load_data()
print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [3]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(32, 32, 3)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(10)
    ]
)

epochsNum = 5
optimizer = keras.optimizers.Adam()
lossFn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metricAcc = keras.metrics.SparseCategoricalAccuracy()

trainWriter = tf.summary.create_file_writer('log/train/')
testWriter = tf.summary.create_file_writer('log/test/')
trainStep = testStep = 0
batchSize = 100

Прежде всего необходимо переписать цикл обучения со встроенного на расширенный

In [4]:
for epoch in range(epochsNum):
    print(f'Epoch № {epoch} ------------------------------------')
    
    # Training loop (Обучающий прогон)
    for i in range(len(xTrain) // batchSize):
        xBatch = xTrain[i * batchSize : (i + 1) * batchSize]
        yBatch = yTrain[i * batchSize : (i + 1) * batchSize]
        with tf.GradientTape() as tape:
            yPred = model(xBatch, training=True)
            loss = lossFn(yBatch, yPred)
            
        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        metricAcc.update_state(yBatch, yPred)
    
    with trainWriter.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar('Accuracy', metricAcc.result(), step=epoch)
    
    print('Train Accuracy: ', metricAcc.result())
    metricAcc.reset_states()
    
    # Test loop (Тестовый прогон)
    for i in range(len(xTest) // batchSize):
        xBatch = xTrain[i * batchSize : (i + 1) * batchSize]
        yBatch = yTrain[i * batchSize : (i + 1) * batchSize]
        yPred = model(xBatch, training=False)
        metricAcc.update_state(yBatch, yPred)

    with testWriter.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar('Accuracy', metricAcc.result(), step=epoch)

            
    print('Test Accuracy: ', metricAcc.result(), '\n')
    metricAcc.reset_states()

Epoch № 0 ------------------------------------
Train Accuracy:  tf.Tensor(0.40764, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.5202, shape=(), dtype=float32) 

Epoch № 1 ------------------------------------
Train Accuracy:  tf.Tensor(0.54452, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.5865, shape=(), dtype=float32) 

Epoch № 2 ------------------------------------
Train Accuracy:  tf.Tensor(0.6, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.6179, shape=(), dtype=float32) 

Epoch № 3 ------------------------------------
Train Accuracy:  tf.Tensor(0.64344, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.6754, shape=(), dtype=float32) 

Epoch № 4 ------------------------------------
Train Accuracy:  tf.Tensor(0.67354, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.6866, shape=(), dtype=float32) 



<h4>Сохранение и загрузка весов</h4>

In [5]:
model.save_weights('modelFolder/') #Сохраняются только веса модели и нам необходимо помнить всю архитектуру модели

In [7]:
model.load_weights('modelFolder/')
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)
model.evaluate(xTest, yTest, batch_size=128, verbose=2)

79/79 - 2s - loss: 1.1043 - accuracy: 0.6285


[1.1042855978012085, 0.6284999847412109]

<h4>Сохранение и загрузка модели целиком</h4>

In [8]:
model.save('fullModelFolder/') #Все сохраняется целиком

INFO:tensorflow:Assets written to: fullModelFolder/assets


In [9]:
model2 = keras.models.load_model('fullModelFolder/')
model2.evaluate(xTest, yTest, batch_size=128, verbose=2)

79/79 - 2s - loss: 1.1043 - accuracy: 0.6285


[1.1042855978012085, 0.6284999847412109]